# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [263]:
!pip install ultralytics

In [264]:
from ultralytics import YOLO
import cv2


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [265]:
model_path = 'YOLO_Model.pt'
model = YOLO(model_path)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [266]:
# getting the video
video_path = 'YOLO_video_cut.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [267]:
# Getting properties of the video
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [268]:
output_path = 'output_video_final.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [271]:
!pwd

/content


In [272]:
!ls

output_video_final.mp4	sample_data  YOLO_Model.pt	 YOLO_Video.mp4
output_video.mp4	sort	     YOLO_video_cut.mp4


In [273]:
top_count = 0
bottom_count = 0
left_count = 0
right_count = 0

top_counted_ids = set()
bottom_counted_ids = set()
left_counted_ids = set()
right_counted_ids = set()

In [274]:
top_line_points = [(int(w * 0.1), int(h * 0.2)), (int(w * 0.9), int(h * 0.2))]
bottom_line_points = [(int(w * 0.1), int(h * 0.8)), (int(w * 0.9), int(h * 0.8))]
left_line_points = [(int(w * 0.2), int(h * 0.1)), (int(w * 0.2), int(h * 0.9))]
right_line_points = [(int(w * 0.8), int(h * 0.1)), (int(w * 0.8), int(h * 0.9))]



In [275]:

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("the freme is empty or video is completed.")
        break

    tracks = model.track(im0, persist=True, show=False)

    for result in tracks:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            obj_id = int(box.id)
            conf = float(box.conf)
            cx = (x1 + x2) // 2  
            cy = (y1 + y2) // 2  

            if cy < top_line_points[0][1] and obj_id not in top_counted_ids:
                top_count += 1
                top_counted_ids.add(obj_id)
            elif cy > bottom_line_points[0][1] and obj_id not in bottom_counted_ids:
                bottom_count += 1
                bottom_counted_ids.add(obj_id)
            elif cx < left_line_points[0][0] and obj_id not in left_counted_ids:
                left_count += 1
                left_counted_ids.add(obj_id)
            elif cx > right_line_points[0][0] and obj_id not in right_counted_ids:
                right_count += 1
                right_counted_ids.add(obj_id)

            label = f'ID: {obj_id} Conf: {conf:.2f}'
            cv2.rectangle(im0, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(im0, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    #  ROI lines
    cv2.line(im0, top_line_points[0], top_line_points[1], (255, 0, 0), 2)
    cv2.line(im0, bottom_line_points[0], bottom_line_points[1], (0, 255, 0), 2)
    cv2.line(im0, left_line_points[0], left_line_points[1], (0, 0, 255), 2)
    cv2.line(im0, right_line_points[0], right_line_points[1], (255, 255, 0), 2)

    cv2.putText(im0, f"Top Exit: {top_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(im0, f"Bottom Exit: {bottom_count}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(im0, f"Left Exit: {left_count}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(im0, f"Right Exit: {right_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.putText(im0, "Rakan Al Rasheed", (w - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 64.7ms
Speed: 2.5ms preprocess, 64.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 49.6ms
Speed: 3.2ms preprocess, 49.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 48.5ms
Speed: 3.0ms preprocess, 48.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 42.1ms
Speed: 4.1ms preprocess, 42.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 40.7ms
Speed: 3.2ms preprocess, 40.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 41.7ms
Speed: 3.8ms preprocess, 41.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 41.0ms
Speed: 3.0ms preprocess, 41.0ms inferenc

https://medium.com/@svr1052000/implementing-yolov8-for-car-counting-84b667adb29a

## Save and Submit Your Work

Submit both the notebook and the output video